Temporary notebook to collect up the functions that I'll use later when I make this official

In [1]:
from collections import defaultdict
import os, sys
import time
import pandas as pd
import GetOldTweets3 as got

def get_new_tweets(t_last_tweet,username = "elonmusk"):
    """Function to scrape the recent tweets of Elon Musk"""
    #t_last_tweet must be pandas Timestamp data
    os.makedirs('tweet_data', exist_ok=True)
    date_str = str(t_last_tweet.date().year)+"-"\
              +str(t_last_tweet.date().month)+"-"\
              +str(t_last_tweet.date().day)
    count = 0
    # Creation of query object                                                                                                                                                                                      
    tweetCriteria = got.manager.TweetCriteria().setUsername(username)\
                                               .setMaxTweets(count)\
                                               .setSince(date_str)
    # Creation of list that contains all tweets                                                                                                                                                                     
    tweets = None
    for ntries in range(5):
        try:
            tweets = got.manager.TweetManager.getTweets(tweetCriteria)
        except SystemExit:
            print("Trying again in 15 minutes.")
            time.sleep(15*60)
        else:
            break
    if tweets is None:
        print("Failed after 5 tries, quitting!")
        exit(1)

    data = defaultdict(list)
    for t in tweets:
        data["username"].append(username)
        data["tweet_id"].append(t.id)
        data["reply_to"].append(t.to)
        data["date"].append(t.date)
        data["retweets"].append(t.retweets)
        data["favorites"].append(t.favorites)
        data["hashtags"].append(list(set(t.hashtags.split())))
        data["mentions"].append(t.mentions)
        data["text"].append(t.text)
        data["permalink"].append(t.permalink)
    if len(data) == 0: #no new tweets
        return None
    else:
        #make a DataFrame out of the scraped tweets
        df = pd.DataFrame(data, columns=["username","tweet_id","reply_to","date","retweets","favorites","hashtags","mentions","text","permalink"])        
        # Convert 'Time' column to datetime and strip time information.
        df['Time'] = pd.to_datetime(df['date'])
        return df

In [2]:
import pandas as pd

def reload_tweet_data(username="elonmusk"):
   #note we'll have to do a .drop and set the 'Time' column to the proper values every time
    df = pd.read_csv('./tweet_data/'+username+'.csv').drop(['Unnamed: 0'],axis='columns')
    df['Time'] = pd.to_datetime(df['Time'])
    return df



In [3]:
import re
def convert_hashtag(text): #renders a hastag into plain text for NLP
    txt_list = re.findall('[A-Z][^A-Z]*', text)
    return map(lambda x:x.lower()," ".join(txt_list)) 

def standardize_text(df, text_field):
    df[text_field] = df[text_field].str.replace(r"http\S+", "")
    df[text_field] = df[text_field].str.replace(r"http", "")
    df[text_field] = df[text_field].str.replace(r"@\S+", "")
    df[text_field] = df[text_field].str.replace(r"RT", "")
    df[text_field] = df[text_field].str.replace(r"[^A-Za-z0-9(),:;!?@\'\`\"\_\n]", " ")
    df[text_field] = df[text_field].str.replace(r"@", "at")
    df[text_field] = df[text_field].str.lower()
    return df

In [ ]:
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.tokenize import RegexpTokenizer
nltk.download('vader_lexicon') #get the bloody lexicon
sid = SentimentIntensityAnalyzer() #returns error if no lexicon
def tweet_word_count(df)
    tokenizer = RegexpTokenizer(r'\w+') #split on words
    df["tokens"] = df["tweet"].apply(tokenizer.tokenize) #returns list of individual words
    df['tweet_length'] = df.apply(lambda row : len(row['tokens']), axis=1) #creates tweet length column
    df.drop(['tokens'],axis='columns')
    
    
def construct_features(tweets):
    """Constructs features from Elon's tweet data"""
    tweets = standardize_text(tweets,"tweet")
    tweets["hashtags"] = tweetsconvert_hastag()
    #Make a rate of sentiment change feature
tweets_clean_df['dSentiment_dTime'] = (tweets_clean_df['Sentiment']-tweets_clean_df['Sentiment'].shift(-1)).fillna(0.)/(tweets_clean_df['delta_Time'])
tweets_clean_df['dSentiment_dTweet'] = (tweets_clean_df['Sentiment']-tweets_clean_df['Sentiment'].shift(-1)).fillna(0.)
